In [18]:
import pandas as pd
import json
import multiprocessing
from multiprocessing import Pool
import numpy as np

In [2]:
def get_data_and_header():
    df = pd.read_csv('../Data/usa_00021.csv')
    drop_list = ['DATANUM', 'SERIAL', 'CBSERIAL', 'PERNUM', 'RACED', 'BPLD', 'LANGUAGED', 'OCC', 'IND', 'HHWT']
    df.drop(drop_list, axis=1, inplace=True)
    header = list(df)
    return df, header

def read_json(f_name):
    with open(f_name) as j:  
        codebook = json.load(j)
    return codebook

In [28]:
def is_skip_case(h):
    # these are basically numeric terms
    return h in ['INCOME', 'HHWT', 'FARMPROD', 'MORTAMT1', 'PROPINSR', 'RENTGRS', 'CONDOFEE', 'MOBLHOME'
                ,'COSTELEC', 'COSTGAS', 'COSTWATR', 'COSTFUEL', 'HHINCOME', 'VALUEH', 'PERWT',
                'MOMLOC', 'POPLOC', 'BIRTHYR', 'RACED', 'YRIMMIG', 'YRSUSA1', 'OCC', 'IND', 'INCTOT',
                'FTOTINC', 'INCSS', 'INCWELFR', 'INCRETIR', 'POVERTY', 'OCCSCORE', 'SEI', 'HWSEI',
                'PRESGL', 'PRENT', 'MOVEDIN', 'RIDERS', 'TRANTIME', 'DEPARTS', 'ROOMS']


# It works well with variables defined within the kernel
# It fails when we call other functions inside the process
# But it won't error out, it just returns a None Type 
def transform_row(row):
    
    new_row = list(row[:])
    
    exceptions = {}
    
    for idx, each in enumerate(row):
        
        h = header[idx]

        if h == 'YEAR':
            new_row[idx] = each
            continue

        key = str(int(each))

        if is_skip_case(h):
            new_row[idx] = int(key) 
            continue
                    
        try:
            new_row[idx] = codes[h][key]

        except:
            print (h, key)
#             new_row[idx] = codes[h][key]

    return new_row

def magic_loop(rows):
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows= pool.map(transform_row, rows)
    pool.close() 
    pool.join()
    df = pd.DataFrame(transformed_rows, columns = header)
    return df

In [4]:
codes = read_json('../Data/codebook.json')
# the full file takes about 3.25 minutes to load into memory on an m5.4xl 
df, header = get_data_and_header()

In [8]:
rows = df.as_matrix()

In [39]:
transformed_df = magic_loop(rows)

In [41]:
transformed_df.to_csv('../Data/transformed_2017.csv', index=False)

In [43]:
!aws s3 cp '../Data/transformed_2017.csv' s3://ep-state-data-raw/Sample-2017/

upload: ../Data/transformed_2017.csv to s3://ep-state-data-raw/Sample-2017/transformed_2017.csv
